The mandatory tour scheduling model selects a tour departure and duration period (and therefore a start and end period as well) for each mandatory tour

In [1]:
import os
import toml
import pandas as pd
#import validation_data_input
import numpy as np
import plotly.express as px
import psrc_theme

import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected" # to show plotly figures in quarto HTML file
pio.templates.default = "simple_white+psrc_color" # set plotly template

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))



In [2]:
# get shared data
%store -r validation_data

#config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

tour_data = validation_data.tours.copy()
m_tour_data = tour_data.loc[tour_data['tour_category']=="mandatory"]
ws_tour_data = m_tour_data.loc[m_tour_data['tour_type'].isin(["work","school"])]

# delete shared data
del validation_data


In [3]:
m_tour_data.source.value_counts()

model results        1693551
survey data             3521
unweighted survey       3521
Name: source, dtype: int64

## tour start time

In [4]:
df_plot = m_tour_data.groupby(['source','start'])[['tour_weight']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="start", y="percentage", color="source",barmode="group",
             title="mandatory tour start time")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis=dict(tickformat=".0%"), xaxis=dict(dtick="1"))
fig.show()

- number of tours with start time later than 8pm

In [5]:
df_plot.loc[df_plot['start']>=20]

,source,start,tour_weight,percentage
15,model results,20.0,5181.000000,0.003059
16,model results,21.0,1647.000000,0.000973
17,model results,22.0,83.000000,0.000049
18,model results,23.0,154.000000,0.000091
34,survey data,20.0,2664.395448,0.001632
35,survey data,21.0,1284.251575,0.000786
36,survey data,22.0,7463.727136,0.004570
37,survey data,23.0,255.980387,0.000157
53,unweighted survey,20.0,9.000000,0.002556
54,unweighted survey,21.0,3.000000,0.000852


In [6]:
df_plot = ws_tour_data.groupby(['source','tour_type','start'])[['tour_weight']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['tour_type','source'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="start", y="percentage", color="source",barmode="group",
             facet_row='tour_type',
             title="work/school tour start time")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis1=dict(tickformat=".0%"), yaxis2=dict(tickformat=".0%", matches=None), xaxis=dict(dtick="1"))
fig.show()

- bars with tour counts fewer than 30

In [7]:
df_plot.loc[df_plot['tour_weight']<30]

,source,tour_type,start,tour_weight,percentage
48,survey data,school,15.0,28.838703,0.000082
70,unweighted survey,school,5.0,5.000000,0.008741
71,unweighted survey,school,6.0,25.000000,0.043706
75,unweighted survey,school,10.0,17.000000,0.029720
76,unweighted survey,school,11.0,13.000000,0.022727
77,unweighted survey,school,12.0,8.000000,0.013986
78,unweighted survey,school,13.0,11.000000,0.019231
79,unweighted survey,school,14.0,7.000000,0.012238
80,unweighted survey,school,15.0,3.000000,0.005245
81,unweighted survey,school,16.0,8.000000,0.013986


- work/school tour count

In [8]:
ws_count = ws_tour_data.groupby(['source','tour_type'])[['tour_weight']].sum().reset_index()
ws_count['trip_weight_2017_2019'] = ws_count['tour_weight'].apply(lambda x: round(x,1))
# df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['trip_weight_2017_2019']. \
#     apply(lambda x: x / float(x.sum()))
ws_count.sort_values('tour_type', ascending=False)

,source,tour_type,tour_weight,trip_weight_2017_2019
1,model results,work,1.114124e+06,1114124.0
3,survey data,work,1.280246e+06,1280246.0
5,unweighted survey,work,2.949000e+03,2949.0
0,model results,school,5.794270e+05,579427.0
2,survey data,school,3.528335e+05,352833.5
4,unweighted survey,school,5.720000e+02,572.0


## tour duration

In [9]:
m_tour_data['duration_cal'] = m_tour_data['end'] - m_tour_data['start']

df_plot = m_tour_data.groupby(['source','duration_cal'])[['tour_weight']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="duration_cal", y="percentage", color="source",barmode="group",
             title="mandatory tour duration")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, yaxis=dict(tickformat=".0%"), xaxis=dict(dtick="1"))
fig.show()

C:\Users\Modeller\AppData\Local\Temp\ipykernel_14032\3604968384.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
ws_tour_data['duration_cal'] = ws_tour_data['end'] - ws_tour_data['start']

df_plot = ws_tour_data.groupby(['source','tour_type','duration_cal'])[['tour_weight']].sum().reset_index()
df_plot['percentage'] = df_plot.groupby(['tour_type','source'], group_keys=False)['tour_weight']. \
    apply(lambda x: x / float(x.sum()))

fig = px.bar(df_plot, x="duration_cal", y="percentage", color="source",barmode="group",
             facet_row='tour_type',
             title="work/school tour duration")
fig.for_each_annotation(lambda a: a.update(text = a.text.split("=")[-1]))
fig.update_layout(height=400, width=700, font=dict(size=11), yaxis1=dict(tickformat=".0%"), yaxis2=dict(tickformat=".0%", matches=None), xaxis=dict(dtick="1"))
fig.show()

- bars with tour counts fewer than 30

In [11]:
df_plot.loc[df_plot['tour_weight']<30]

,source,tour_type,duration_cal,tour_weight,percentage
18,model results,school,18.0,14.0,0.000024
72,unweighted survey,school,0.0,10.0,0.017483
73,unweighted survey,school,1.0,10.0,0.017483
74,unweighted survey,school,2.0,14.0,0.024476
75,unweighted survey,school,3.0,24.0,0.041958
76,unweighted survey,school,4.0,22.0,0.038462
77,unweighted survey,school,5.0,26.0,0.045455
78,unweighted survey,school,6.0,29.0,0.050699
83,unweighted survey,school,11.0,19.0,0.033217
84,unweighted survey,school,12.0,16.0,0.027972
